<a href="https://colab.research.google.com/github/AzulBarr/Aprendizaje-Automatico/blob/main/TPs/tp2/RF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Carga de librerias y modelos

In [5]:
#!pip install ebooklib beautifulsoup4 pandas
!pip install -U transformers huggingface_hub datasets accelerate sentence-transformers

  Using cached huggingface_hub-1.1.6-py3-none-any.whl.metadata (13 kB)


In [6]:
import torch
from google.colab import drive
from transformers import BertTokenizer, BertModel
import numpy as np
import pandas as pd
import re
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score,confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

In [7]:
model_name = "bert-base-multilingual-cased"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

# Definición de funciones

In [38]:
def normalize_text_X(t):
    # Convertir a minúsculas y quitar puntuación
    t = re.sub(r'(?<!\w)A\.C\.(?!\w)', 'a c', t, flags=re.IGNORECASE)
    t = re.sub(r'(?<!\w)P\.S\.(?!\w)', 'p s', t, flags=re.IGNORECASE)
    t = t.lower()
    t = re.sub(r'[\u200b-\u200f\uFEFF]', '', t)
    t = re.sub(r"[^a-záéíóúüñ0-9' ]+", ' ', t)
    #t = re.sub(r'p\s*\.?\s*s\s*\.?', 'ps', t, flags=re.IGNORECASE)
    #t = re.sub(r'a\s*\.?\s*c\s*\.?', 'ac', t, flags=re.IGNORECASE)
    t = re.sub(r'\s+', ' ', t).strip()
    return t

In [25]:
print(normalize_text_X('hola P.S. PS P.S'))
print(normalize_text_y('hola P.S. PS P.S'))

hola ps ps p s
hola PS PS P.S


In [39]:
def normalize_text_y(t):
    t = re.sub(r'[\u200b-\u200f\uFEFF]', '', t)
    t = re.sub(r'\.\.\.', ' ', t)
    t = re.sub(r'\.\.', ' ', t)
    t = re.sub(r'\,\,', ' ', t)
    t = re.sub(r"[^a-zA-ZáéíóúüñÁÉÍÓÚ0-9¿?,.' ]+", ' ', t)
    #t = re.sub(r'p\s*\.?\s*s\s*\.?', 'PS', t, flags=re.IGNORECASE)
    #t = re.sub(r'a\s*\.?\s*c\s*\.?', 'AC', t, flags=re.IGNORECASE)
    t = re.sub(r'(?<!\w)P\.S\.(?!\w)', 'P S', t, flags=re.IGNORECASE)
    t = re.sub(r'(?<!\w)A\.C\.(?!\w)', 'A C', t, flags=re.IGNORECASE)
    t = re.sub(r'\s+', ' ', t).strip()
    return t

In [15]:
def crearDataSetRFSinEtiquetas(parrafo): #CAMBIAR
  data_set_RF_sin_etiquetas = pd.DataFrame(columns = ['instancia_id', 'token', 'token_id', 'posicion_frase',
                                        'categoria_gramatical', 'distancia_al_final',
                                        'id_anterior', 'id_siguiente', 'es_principio',
                                        'es_medio', 'es_final', 'forma_parte'])

  instancia_ids = []
  tokens = []
  token_ids = []
  posiciones_parrafo = []
  categorias_gramaticales = []
  distancias_al_final = []
  ids_anteriores = []
  ids_siguientes = []
  son_principio = []
  son_medio = []
  son_final = []
  forman_parte = []

  for k, parrafo in enumerate(parrafo):
    instancia_id = k
    token_siguiente = -1
    token_anterior = -1
    palabras = parrafo.split()

    inicio_pregunta = False

    for i, palabra in enumerate(palabras):
      #categoria = indice_categoria_stanza(palabra)
      categoria = 0
      tokens_de_palabra = tokenizer.tokenize(palabra)

      for j, token in enumerate(tokens_de_palabra):
        id = tokenizer.convert_tokens_to_ids(token)
        tokens.append(token)
        token_ids.append(id)
        posiciones_parrafo.append(i)
        categorias_gramaticales.append(categoria)
        distancias_al_final.append(len(palabras) - i)

        ids_anteriores.append(token_anterior)
        token_anterior = id

        n_tok = len(tokens_de_palabra)
        if j != n_tok - 1:
          id_sig  = tokenizer.convert_tokens_to_ids(tokens_de_palabra[j + 1])
          ids_siguientes.append(id_sig)
        else:
          if i != len(palabras) - 1:
            token_siguiente = tokenizer.convert_tokens_to_ids(tokenizer.tokenize(palabras[i + 1]))[0]
            ids_siguientes.append(token_siguiente)
          else:
            ids_siguientes.append(-1)

        es_medio_id = 0
        es_final_id = 0
        es_principio_id = 0
        if j == 0:
          es_principio_id = 1
        elif j == n_tok - 1:
          es_final_id = 1
        else:
          es_medio_id = 1
        son_principio.append(es_principio_id)
        son_medio.append(es_medio_id)
        son_final.append(es_final_id)

        forma_parte_id = 0
        if n_tok != 1:
          forma_parte_id = 1
        forman_parte.append(forma_parte_id)

        instancia_ids.append(instancia_id)

  data_set_RF_sin_etiquetas['instancia_id'] = instancia_ids
  data_set_RF_sin_etiquetas['token'] = tokens
  data_set_RF_sin_etiquetas['token_id'] = token_ids
  data_set_RF_sin_etiquetas['posicion_frase'] = posiciones_parrafo
  data_set_RF_sin_etiquetas['categoria_gramatical'] = categorias_gramaticales
  data_set_RF_sin_etiquetas['distancia_al_final'] = distancias_al_final
  data_set_RF_sin_etiquetas['id_anterior'] = ids_anteriores
  data_set_RF_sin_etiquetas['id_siguiente'] = ids_siguientes
  data_set_RF_sin_etiquetas['es_principio'] = son_principio
  data_set_RF_sin_etiquetas['es_medio'] = son_medio
  data_set_RF_sin_etiquetas['es_final'] = son_final
  data_set_RF_sin_etiquetas['forma_parte'] = forman_parte

  return data_set_RF_sin_etiquetas

In [32]:
def crearDataSetRFConEtiquetas(parrafos):
  data_set_RF_con_etiquetas = pd.DataFrame(columns = ['palabra_default', 'instancia_id', 'token', 'token_id', 'posicion_frase',
                                        'categoria_gramatical', 'distancia_al_final',
                                        'id_anterior', 'id_siguiente', 'es_principio',
                                        'es_medio', 'es_final', 'forma_parte', 'punt_inicial', 'punt_final', 'capitalización'])

  palabras_default = []

  instancia_ids = []
  tokens = []
  token_ids = []
  posiciones_parrafo = []
  categorias_gramaticales = []
  distancias_al_final = []
  ids_anteriores = []
  ids_siguientes = []
  son_principio = []
  son_medio = []
  son_final = []
  forman_parte = []
  puntuaciones_iniciales = []
  puntuaciones_finales = []
  capitalizaciones = []

  datos_limpios = parrafos['limpio']
  datos_default = parrafos['default']
  for k, parrafo in enumerate(datos_limpios):
    instancia_id = k
    token_siguiente = -1
    token_anterior = -1
    palabras = parrafo.split()

    inicio_pregunta = False


    palabras_d = datos_default.iloc[k].split()
    palabras_d = [subpalabra
                for palabra in palabras_d
                for subpalabra in (palabra.split('.') if (palabra.count('.') > 1 or ('.' in palabra and palabra[-1] != '.' and palabra[0] != '.')) else [palabra])]
    palabras_d = [subpalabra
                for palabra in palabras_d
                for subpalabra in (palabra.split(',') if (palabra.count(',') > 1 or (',' in palabra and palabra[-1] != ',' and palabra[0] != ',')) else [palabra])]

    palabras_d = [subpalabra
                for palabra in palabras_d
                for subpalabra in (palabra.split('?') if (palabra.count('?') > 1 or ('?' in palabra and palabra[-1] != '?' and palabra[0] != '?')) else [palabra])]

    palabras_d = [subpalabra
                for palabra in palabras_d
                for subpalabra in (palabra.split('¿') if (palabra.count('¿') > 1 or ('¿' in palabra and palabra[-1] != '¿' and palabra[0] != '¿')) else [palabra])]
    palabras_d = [palabra for palabra in palabras_d if (palabra != '')]


    npal = 0
    for i, palabra in enumerate(palabras):

      palabra_default = ' '

      if palabra == "'":
        continue

      while palabras_d[npal] == '?' or palabras_d[npal] == '¿' or palabras_d[npal] == '.' or palabras_d[npal] == ',' :
        if palabras_d[npal] == '¿':
          inicio_pregunta = True
        npal += 1

      if palabras_d[npal] != '?' and palabras_d[npal] != '¿' and palabras_d[npal] != '.' and palabras_d[npal] != ',':
        palabra_default = palabras_d[npal]

      if palabra_default[0] == "¿":
        inicio_pregunta = True

      if palabra_default[-1] == "?":
        puntuacion_final = 3
      elif palabra_default[-1] == ".":
        puntuacion_final = 1
      elif palabra_default[-1] == ",":
        puntuacion_final = 2
      else:
        if npal != len(palabras_d)-1:
          if palabras_d[npal+1] == '?':
            puntuacion_final = 3
          elif palabras_d[npal+1] == '.':
            puntuacion_final = 1
          elif palabras_d[npal+1] == ',':
            puntuacion_final = 2
          else:
            puntuacion_final = 0
        else:
          puntuacion_final = 0

      categoria = 0
      tokens_de_palabra = tokenizer.tokenize(palabra)

      if palabra_default.islower():
        capitalizacion = 0
      elif palabra_default.istitle():
        capitalizacion = 1
      elif palabra_default.isupper():
        capitalizacion = 3
      else:
        capitalizacion = 2

      npal += 1
      for j, token in enumerate(tokens_de_palabra):
        id = tokenizer.convert_tokens_to_ids(token)
        tokens.append(token)
        token_ids.append(id)
        posiciones_parrafo.append(i)
        categorias_gramaticales.append(categoria)
        distancias_al_final.append(len(palabras) - i)

        ids_anteriores.append(token_anterior)
        token_anterior = id

        n_tok = len(tokens_de_palabra)
        if j != n_tok - 1:
          id_sig  = tokenizer.convert_tokens_to_ids(tokens_de_palabra[j + 1])
          ids_siguientes.append(id_sig)
        else:
          if i != len(palabras) - 1:
            token_siguiente = tokenizer.convert_tokens_to_ids(tokenizer.tokenize(palabras[i + 1]))[0]
            ids_siguientes.append(token_siguiente)
          else:
            ids_siguientes.append(-1)

        es_medio_id = 0
        es_final_id = 0
        es_principio_id = 0
        if j == 0:
          es_principio_id = 1
        elif j == n_tok - 1:
          es_final_id = 1
        else:
          es_medio_id = 1
        son_principio.append(es_principio_id)
        son_medio.append(es_medio_id)
        son_final.append(es_final_id)

        forma_parte_id = 0
        if n_tok != 1:
          forma_parte_id = 1
        forman_parte.append(forma_parte_id)

        if inicio_pregunta:
          puntuaciones_iniciales.append(1)
          inicio_pregunta = False
        else:
          puntuaciones_iniciales.append(0)

        if j == n_tok - 1:
          puntuaciones_finales.append(puntuacion_final)
        else:
          puntuaciones_finales.append(0)

        capitalizaciones.append(capitalizacion)
        instancia_ids.append(instancia_id)
        palabras_default.append(palabra_default)
      print(instancia_id)


  data_set_RF_con_etiquetas['palabra_default'] = palabras_default
  data_set_RF_con_etiquetas['instancia_id'] = instancia_ids
  data_set_RF_con_etiquetas['token'] = tokens
  data_set_RF_con_etiquetas['token_id'] = token_ids
  data_set_RF_con_etiquetas['posicion_frase'] = posiciones_parrafo
  data_set_RF_con_etiquetas['categoria_gramatical'] = categorias_gramaticales
  data_set_RF_con_etiquetas['distancia_al_final'] = distancias_al_final
  data_set_RF_con_etiquetas['id_anterior'] = ids_anteriores
  data_set_RF_con_etiquetas['id_siguiente'] = ids_siguientes
  data_set_RF_con_etiquetas['es_principio'] = son_principio
  data_set_RF_con_etiquetas['es_medio'] = son_medio
  data_set_RF_con_etiquetas['es_final'] = son_final
  data_set_RF_con_etiquetas['forma_parte'] = forman_parte
  data_set_RF_con_etiquetas['punt_inicial'] = puntuaciones_iniciales
  data_set_RF_con_etiquetas['punt_final'] = puntuaciones_finales
  data_set_RF_con_etiquetas['capitalización'] = capitalizaciones

  return data_set_RF_con_etiquetas

# Carga de datos y creación de data set

In [ ]:
path = 'https://raw.githubusercontent.com/AzulBarr/Aprendizaje-Automatico/main/TPs/tp2'
libro1 = '/Harry_Potter_y_el_caliz_de_fuego_J_K_Rowling.epub'
path = path + libro1

In [ ]:
!wget -O libro1.epub $path

In [ ]:
#parrafos = convertir_epub_a_pd('libro1.epub')

In [ ]:
#dataSet_RF_con_etiquetas = crearDataSetRFConEtiquetas(parrafos)
#dataSet_RF_sin_etiquetas = crearDataSetRFSinEtiquetas(parrafos['limpio'])

In [ ]:
#dataSet_RF_con_etiquetas = crearDataSetRFConEtiquetas(parrafos)
#dataSet_RF_sin_etiquetas = crearDataSetRFSinEtiquetas(parrafos['limpio'])

In [ ]:
path = 'https://raw.githubusercontent.com/AzulBarr/Aprendizaje-Automatico/main/TPs/tp2'
datos = '/dataSet_RF_con_etiquetas.csv'
path = path + datos

In [ ]:
#!wget -O dataSet_RF_con_etiquetas.csv $path

In [ ]:
#dataSet_RF_con_etiquetas_cg = pd.read_csv('dataSet_RF_con_etiquetas.csv')

In [26]:
parrafos_final = pd.read_csv('dataset_parrafos_total3.csv')

In [28]:
parrafos_final[12294:12295]

,default,limpio
12294,"Aramis releyó entonces la carta, y se dio cuen...",aramis releyó entonces la carta y se dio cuent...


In [40]:
parrafos_final = pd.read_csv('dataset_parrafos_total3.csv')
parrafos_final = parrafos_final.dropna()
parrafos_final['limpio'] = parrafos_final['limpio'].apply(normalize_text_X)
parrafos_final['default'] = parrafos_final['default'].apply(normalize_text_y)
parrafos_final.reset_index(drop=True)
parrafos_final.drop(index=[7764, 9497, 9498], inplace=True) #palabra.¿palabra parrafo raro, parrafo raro
parrafos_final.reset_index(drop=True)

,default,limpio
0,Entonces es que es así como debo llamarme repu...,entonces es que es así como debo llamarme repu...
1,"Pero, Heidi, las cosas no son tan simples",pero heidi las cosas no son tan simples
2,¿Entonces qué es?,entonces qué es
3,¿No te he prohibido terminantemente que vayas ...,no te he prohibido terminantemente que vayas a...
4,"Y luego, cuando nos encontramos desgraciados y...",y luego cuando nos encontramos desgraciados y ...
...,...,...
243607,Soy Donna.,soy donna
243608,"Cuidado con llamarme pequeño, o te",cuidado con llamarme pequeño o te
243609,La mitad se puso de pie.,la mitad se puso de pie
243610,Tienes una acreción de placenta.,tienes una acreción de placenta


In [41]:
dataSet_RF_con_etiquetas_final = crearDataSetRFConEtiquetas(parrafos_final)

Se truncaron las últimas líneas 5000 del resultado de transmisión.
243285
243285
243285
243285
243285
243285
243285
243285
243285
243285
243285
243285
243285
243285
243285
243285
243285
243285
243285
243285
243285
243285
243285
243285
243285
243285
243285
243285
243285
243285
243285
243285
243285
243285
243285
243285
243285
243285
243285
243285
243285
243285
243285
243285
243285
243286
243286
243286
243286
243286
243286
243286
243286
243286
243286
243286
243286
243286
243286
243287
243287
243287
243288
243288
243288
243288
243288
243288
243288
243288
243288
243288
243288
243288
243288
243288
243288
243289
243289
243289
243289
243289
243289
243289
243289
243289
243289
243289
243289
243289
243289
243289
243289
243289
243289
243289
243289
243289
243289
243289
243289
243289
243289
243289
243289
243289
243289
243289
243289
243289
243289
243289
243290
243290
243290
243290
243290
243290
243290
243290
243290
243290
243290
243290
243290
243290
243290
243290
243290
243290
243290
243291
243291
24

In [43]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [44]:
dataSet_RF_con_etiquetas_final.to_csv('/content/drive/MyDrive/dataSet_RF_con_etiquetas_final.csv', index=False)


In [ ]:
#dataSet_RF_con_etiquetas_final = crearDataSetRFConEtiquetas(parrafos_final)
dataSet_RF_con_etiquetas_final = pd.read_csv('/content/drive/MyDrive/dataSet_RF_con_etiquetas_final.csv')

## Etiqueta 1: capitalización
* 0: todo en minúsculas (ej: “hola”)
* 1: primera letra en mayúscula (ej: “Hola”) (incluye palabras de 1 letra)
* 2: algunas (pero no todas) letras en mayúscula (ej: “McDonald's”  “iPhone”)
* 3: todo en mayúsculas (ej.: “ONU”, “NASA”, “UBA”) (más de una letra)

## Etiqueta 2: puntuación inicial
* 0: sin puntuación.
* 1: "?"

## Etiqueta 3: puntuación final
* 0: sin puntuación.
* 1: "."
* 2: ","
* 3: "?"

# Separación en train y test

In [ ]:
train = dataSet_RF_con_etiquetas[dataSet_RF_con_etiquetas['instancia_id'] <= 2972]
test = dataSet_RF_con_etiquetas[dataSet_RF_con_etiquetas['instancia_id'] > 2972]

In [ ]:
train_cg = dataSet_RF_con_etiquetas_cg[dataSet_RF_con_etiquetas_cg['instancia_id'] <= 2972]
test_cg = dataSet_RF_con_etiquetas_cg[dataSet_RF_con_etiquetas_cg['instancia_id'] > 2972]

In [ ]:
X_train = train.drop(columns=['palabra_default', 'token', 'categoria_gramatical','capitalización', 'punt_inicial', 'punt_final'])
y_train = train[['capitalización', 'punt_inicial', 'punt_final']]
X_test = test.drop(columns=['palabra_default', 'token', 'categoria_gramatical', 'capitalización', 'punt_inicial', 'punt_final'])
y_test = test[['capitalización', 'punt_inicial', 'punt_final']]

In [ ]:
X_train_cg = train_cg.drop(columns=['palabra_default', 'token', 'capitalización', 'punt_inicial', 'punt_final'])
y_train_cg = train_cg[['capitalización', 'punt_inicial', 'punt_final']]
X_test_cg = test_cg.drop(columns=['palabra_default', 'token',  'capitalización', 'punt_inicial', 'punt_final'])
y_test_cg = test_cg[['capitalización', 'punt_inicial', 'punt_final']]

# Sin etiqueta gramatical


## Random Forest para etiqueta 1

In [ ]:
y_train1 = y_train['capitalización']
y_test1 = y_test['capitalización']

In [ ]:
model = RandomForestClassifier(n_estimators=200, max_depth=None, random_state=42, min_samples_split=10)
model.fit(X_train, y_train1)

In [ ]:
y_pred1 = model.predict(X_test)

In [ ]:
f1_score(y_test1, y_pred1, average = "macro")

In [ ]:
cm1 = confusion_matrix(y_test1,y_pred1, labels=[0,1,2,3])

In [ ]:
sns.heatmap(cm1, annot=True, fmt="d", cmap="Blues")
plt.xlabel("Predicho")
plt.ylabel("Real")
plt.title("Matriz de confusión para capitalización sin CG")   # ← Título

plt.show()


## Random Forest para etiqueta 2

In [ ]:
y_train2 = y_train['punt_inicial']
y_test2 = y_test['punt_inicial']

In [ ]:
model = RandomForestClassifier(n_estimators=200, max_depth=None, random_state=42, min_samples_split=10)
model.fit(X_train, y_train2)

In [ ]:
y_pred2 = model.predict(X_test)

In [ ]:
f1_score(y_test2, y_pred2, average = "macro")

In [ ]:
cm2 = confusion_matrix(y_test2, y_pred2, labels=[0,1])

In [ ]:
sns.heatmap(cm2, annot=True, fmt="d", cmap="Blues")
plt.xlabel("Predicho")
plt.ylabel("Real")
plt.title('Matriz de confusión para punt_inicial sin CG')

plt.show()

## Random Forest para etiqueta 3

In [ ]:
y_train3 = y_train['punt_final']
y_test3 = y_test['punt_final']

In [ ]:
model = RandomForestClassifier(n_estimators=200, max_depth=None, random_state=42, min_samples_split=10)
model.fit(X_train, y_train3)

In [ ]:
y_pred3 = model.predict(X_test)

In [ ]:
f1_score(y_test3, y_pred3, average = "macro")

In [ ]:
cm3 = confusion_matrix(y_test3, y_pred3, labels=[0,1,2,3])

In [ ]:
sns.heatmap(cm3, annot=True, fmt="d", cmap="Blues")
plt.xlabel("Predicho")
plt.ylabel("Real")
plt.title('Matriz de confusión para punt_final sin CG')
plt.show()

# Con etiqueta gramatical


## Random Forest para etiqueta 1

In [ ]:
y_train4_cg = y_train_cg['capitalización']
y_test4_cg = y_test_cg['capitalización']

In [ ]:
model = RandomForestClassifier(n_estimators=200, max_depth=None, random_state=42, min_samples_split=10)
model.fit(X_train_cg, y_train4_cg)

In [ ]:
y_pred4_cg = model.predict(X_test_cg)

In [ ]:
f1_score(y_test4_cg, y_pred4_cg, average = "macro")

In [ ]:
cm4 = confusion_matrix(y_test4_cg, y_pred4_cg, labels=[0,1,2,3])

In [ ]:
sns.heatmap(cm4, annot=True, fmt="d", cmap="Blues")
plt.xlabel("Predicho")
plt.ylabel("Real")
plt.title('Matriz de confusión para capitalización con CG')

plt.show()

## Random Forest para etiqueta 2

In [ ]:
y_train5_cg = y_train_cg['punt_inicial']
y_test5_cg = y_test_cg['punt_inicial']

In [ ]:
model = RandomForestClassifier(n_estimators=200, max_depth=None, random_state=42, min_samples_split=10)
model.fit(X_train_cg, y_train5_cg)

In [ ]:
y_pred5_cg = model.predict(X_test_cg)

In [ ]:
f1_score(y_test5_cg, y_pred5_cg, average = "macro")

In [ ]:
cm5 = confusion_matrix(y_test5_cg, y_pred5_cg, labels=[0,1,2,3])

In [ ]:
sns.heatmap(cm5, annot=True, fmt="d", cmap="Blues")
plt.xlabel("Predicho")
plt.ylabel("Real")
plt.title('Matriz de confusión para punt_inicial con CG')

plt.show()

## Random Forest para etiqueta 3

In [ ]:
y_train6_cg = y_train_cg['punt_final']
y_test6_cg = y_test_cg['punt_final']

In [ ]:
model = RandomForestClassifier(n_estimators=200, max_depth=None, random_state=42, min_samples_split=10)
model.fit(X_train_cg, y_train6_cg)

In [ ]:
y_pred6_cg = model.predict(X_test_cg)

In [ ]:
f1_score(y_test6_cg, y_pred6_cg, average = "macro")

In [ ]:
cm6 = confusion_matrix(y_test6_cg, y_pred6_cg, labels=[0,1,2,3])

In [ ]:
sns.heatmap(cm6, annot=True, fmt="d", cmap="Blues")
plt.xlabel("Predicho")
plt.ylabel("Real")
plt.title('Matriz de confusión para punt_final con CG')
plt.show()

### FINAL

In [46]:
dataSet_RF_con_etiquetas_final.shape

(6687158, 16)

In [48]:
train_final = dataSet_RF_con_etiquetas_final[dataSet_RF_con_etiquetas_final['instancia_id'] < 219283]
test_final = dataSet_RF_con_etiquetas_final[dataSet_RF_con_etiquetas_final['instancia_id'] >= 219283]

In [49]:
X_train_final = train_final.drop(columns=['palabra_default', 'token', 'categoria_gramatical','capitalización', 'punt_inicial', 'punt_final'])
y_train_final = train_final[['capitalización', 'punt_inicial', 'punt_final']]
X_test_final = test_final.drop(columns=['palabra_default', 'token', 'categoria_gramatical', 'capitalización', 'punt_inicial', 'punt_final'])
y_test_final = test_final[['capitalización', 'punt_inicial', 'punt_final']]

In [50]:
y_train1_final = y_train_final['capitalización']
y_test1_final = y_test_final['capitalización']

In [ ]:
model = RandomForestClassifier(n_estimators=200, max_depth=None, random_state=42, min_samples_split=10)
model.fit(X_train_final, y_train1_final)

In [ ]:
y_pred1_final = model.predict(X_test_final)

In [ ]:
f1_score(y_test1_final, y_pred1_final, average = "macro")

In [ ]:
cm1_final = confusion_matrix(y_test1_final,y_pred1_final, labels=[0,1,2,3])

In [ ]:
sns.heatmap(cm1_final, annot=True, fmt="d", cmap="Blues")
plt.xlabel("Predicho")
plt.ylabel("Real")
plt.title("Matriz de confusión para capitalización sin CG")   # ← Título

plt.show()